# Importing Libraries

In [ ]:
import os
import math
import spacy
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from PIL import Image
from torch import nn
from matplotlib import pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision.models import vgg16, VGG16_Weights
from nltk.translate.bleu_score import sentence_bleu

spacy_eng = spacy.load("en_core_web_sm")

In [ ]:
!pip install torchsummary
!pip install pycocoevalcap

from torchsummary import summary
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

### Setting the Device

In [ ]:
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('Device:', device)

# Preparing the Flikr30k Dataset
## Custom Vocabulary

In [ ]:
class Vocabulary:
    def __init__(self, freq_threshold):
        self.freq_threshold = freq_threshold
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {v: k for k, v in self.itos.items()}
    
    def __len__(self):
        return len(self.itos)
  
    def build_vocab(self, sentence_list):
        freqs = {}
        idx = 4

        for sentence in sentence_list:
            sentence = str(sentence)

            for word in self.tokenize(sentence):
                freqs[word] = freqs.get(word, 0) + 1

                if freqs[word] == self.freq_threshold:
                    self.itos[idx] = word
                    self.stoi[word] = idx
                    
                    idx += 1

    def numericalize(self, sentence):
        tokens = self.tokenize(sentence)
        result = []

        for token in tokens:
            result.append(self.stoi.get(token, self.stoi["<UNK>"]))

        return result
    
    @staticmethod
    def tokenize(sentence):
        return [token.text.lower() for token in spacy_eng.tokenizer(str(sentence))]

## Custom Dataset

In [ ]:
class Flickr(Dataset):
    def __init__(self, root_dir, caption_path, transform, freq_threshold=5):
        self.freq_threshold = freq_threshold
        self.transform = transform
        self.root_dir = root_dir
    
        self.df = pd.read_csv(caption_path, delimiter='|')
        
        self.images = self.df['image_name']
        self.captions = self.df[' comment']
        
        self.vocab = Vocabulary(freq_threshold)
        
        self.vocab.build_vocab(self.captions.tolist())
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        max_seq_length = 50
        
        image, caption = self.images[index], self.captions[index]
        
        image = Image.open(os.path.join(self.root_dir, image)).convert("RGB")
        
        image = self.transform(image)
        
        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        
        numericalized_caption += self.vocab.numericalize(caption)
        
        numericalized_caption.append(self.vocab.stoi["<EOS>"])
        
        if len(numericalized_caption) > max_seq_length:
            numericalized_caption = numericalized_caption[: max_seq_length]
        else:
            numericalized_caption += [self.vocab.stoi["<PAD>"]] * (max_seq_length - len(numericalized_caption))
        
        return image, torch.tensor(numericalized_caption)
    
    def get_label(self, index):
        _, caption = self[index]
    
        label = [self.vocab.itos[token] for token in caption.tolist()][1: -1]

        return ' '.join(label)
    
    def captions_list(self):
        return self.captions.tolist()
        

## Custom Caption Collat

In [ ]:
class CapCollat:
    def __init__(self, pad_seq, batch_first=False):
        self.pad_seq = pad_seq
        self.batch_first = batch_first
  
    def __call__(self, batch):
        images = [item[0].unsqueeze(0) for item in batch]
        images = torch.cat(images, dim=0)

        target_caps = [item[1] for item in batch]
        target_caps = pad_sequence(target_caps,
                                   batch_first=self.batch_first,
                                   padding_value=self.pad_seq)
        
        return images, target_caps

## Custom Scorer 

In [ ]:
class Scorer():
    def __init__(self, references, candidates):
        self.references = references
        self.candidates = candidates

        self.word_based_scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(),"METEOR"),
            (Rouge(), "ROUGE_L"),
            (Cider(), "CIDEr"),
            ]

    def compute_scores(self):
        total_scores = {}

        for scorer, method in self.word_based_scorers:
            score, _ = scorer.compute_score(self.references, self.candidates)
    
            if type(method) is list:
                total_scores["Bleu_1"] = score[0]
                total_scores["Bleu_2"] = score[1]
                total_scores["Bleu_3"] = score[2]
                total_scores["Bleu_4"] = score[3]

            else:
                total_scores[method] = score

        return total_scores

## Loading and Testing the Dataset 

In [ ]:
root_folder = "/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/"
csv_file = "/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv"

transform = T.Compose([
                T.Resize((256, 256), interpolation=T.InterpolationMode.BILINEAR),
                T.CenterCrop((224, 224)),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ])

batch_size = 37
num_workers = 2
freq_threshold = 5
batch_first = True
pin_memory = True

dataset = Flickr(root_folder, csv_file, transform, freq_threshold)
all_labels = dataset.captions_list()
pad_idx = dataset.vocab.stoi["<PAD>"]

data_size = len(dataset)
train_size = int(0.9 * data_size)
val_size = data_size - train_size

val_set_start = data_size - val_size - 1
val_set_end = data_size - 1

train_set, val_set = torch.utils.data.Subset(dataset, range(0, train_size)), torch.utils.data.Subset(dataset, range(train_size, data_size))

train_loader = DataLoader(train_set,
                            batch_size=batch_size,
                            pin_memory=pin_memory,
                            num_workers=num_workers,
                            shuffle=True,
                            collate_fn=CapCollat(pad_seq=pad_idx, batch_first=batch_first))

val_loader = DataLoader(val_set,
                            batch_size=batch_size,
                            pin_memory=pin_memory,
                            num_workers=num_workers,
                            shuffle=False,
                            collate_fn=CapCollat(pad_seq=pad_idx, batch_first=batch_first))

### Setting Vocabulary

In [ ]:
vocab = dataset.vocab
vocab_size = len(vocab)

### Testing Dataset

In [ ]:
for _ in range(10):
    idx = np.random.randint(0, data_size - 1)

    image, _ = dataset[idx]
    
    label = all_labels[idx]

    image = image.permute(1,2,0)
    
    plt.title(label)
    plt.imshow(image)

    plt.show()

# Encoder:

In [ ]:
class Encoder(nn.Module):
        def __init__(self, model, embed_size: int):
            super(Encoder, self).__init__() 
            
            for param in model.parameters():
                param.requires_grad = False
                
            feature_extractor = list(model.children())[:-1]
                
            embedding_layer = nn.Linear(512 * 7 * 7, embed_size)

            self.encoder = nn.Sequential(*feature_extractor,
                                        nn.Flatten(),
                                        embedding_layer)
        
        def forward(self, image):
            encoded_image = self.encoder(image)
    
            return encoded_image

# Decoder:

In [ ]:
class Decoder(nn.Module):
    class Attention(nn.Module):
        def __init__(self, hidden_size: int):
            super(Decoder.Attention, self).__init__()
            self.hidden_size = hidden_size

            self.fc1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
            self.fc2 = nn.Linear(self.hidden_size*2, self.hidden_size, bias=False)

        def forward(self, hidden_states):
            score_first_part = self.fc1(hidden_states)

            h_t = hidden_states[:,-1,:]

            score = torch.bmm(score_first_part, h_t.unsqueeze(2)).squeeze(2)

            attention_weights = F.softmax(score, dim=1)

            context_vector = torch.bmm(hidden_states.permute(0,2,1), attention_weights.unsqueeze(2)).squeeze(2)

            pre_activation = torch.cat((context_vector, h_t), dim=1)

            attention_vector = self.fc2(pre_activation)
            attention_vector = torch.tanh(attention_vector)

            return attention_vector, attention_weights
        
    def __init__(self, model, embed_size: int, vocab_size: int, hidden_size: int, num_layers: int, attention: bool =False):
        super(Decoder, self).__init__()

        self.attention = attention

        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.features2hidden = nn.Linear(embed_size, hidden_size)

        self.model = model(embed_size, hidden_size, num_layers, batch_first=True)

        if self.attention:
            self.attn = self.Attention(hidden_size=hidden_size)
            
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions):
        captions_embed = self.embedding(captions)

        initial_hidden_state = self.features2hidden(features).unsqueeze(0).repeat(self.model.num_layers, 1, 1)

        output, _ = self.model(captions_embed, initial_hidden_state)

        if self.attention:
            output, _ = self.attn(output)

        output = self.linear(output)

        return output

# Encoder-Decoder:

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder):
        super(EncoderDecoder, self).__init__()
    
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, images, captions):
        x = self.encoder(images)
        x = self.decoder(x, captions)
        
        return x
    
    def caption(self, image, vocabulary, maxlength=50):
        result_caption = []

        with torch.no_grad():
            x = self.encoder(image).unsqueeze(0)
            states = None

            for _ in range(maxlength):
                hiddens, states = self.decoder.model(x, states)
                output = self.decoder.linear(hiddens.squeeze(0))
                predicted = output.argmax(1)
                result_caption.append(predicted.item())
                x = self.decoder.embedding(predicted).unsqueeze(0)

                if vocabulary.itos[predicted.item()] == "<EOS>":
                    break

        return [vocabulary.itos[i] for i in result_caption]


# Custom Trainer:

In [ ]:
class Trainer:
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, num_epochs, save_path = None):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

        self.train_loader = train_loader
        self.val_loader = val_loader
        
        self.num_epochs = num_epochs

        self.save_path = save_path

        self.train_losses = []
        self.val_losses = []

        self.bleu1_scores = []
        self.bleu2_scores = []
        self.bleu3_scores = []
        self.bleu4_scores = []

        self.cumulative_bleu_scores = []

        self.cider_scores = []

        self.meteor_scores = []

        self.rougel_scores = []

    def train(self):
        for epoch in range(self.num_epochs):
            self.model.train()
            
            running_loss = 0.0

            for images, captions in train_loader:
                images, captions = images.to(device), captions.to(device)
                
                train_score = self.model(images, captions)

                self.optimizer.zero_grad()
                
                loss = self.criterion(train_score.view(-1, vocab_size), captions.view(-1))
                running_loss += loss.item()
                
                loss.backward()
                
                self.optimizer.step()
                
            train_loss = running_loss / len(train_loader)
            self.train_losses.append(train_loss)
            
            with torch.no_grad():
                self.model.eval()

                idx = val_set_start
                
                references = {}
                candidates = {}
                
                val_iter = iter(val_loader)
                
                running_loss = 0.0
                
                while idx < val_set_end:
                    if (idx - val_set_start) % batch_size == 0:
                        batch = next(val_iter)

                    images, captions = batch

                    images, captions = images.to(device), captions.to(device)

                    val_score = self.model(images, captions)

                    loss = self.criterion(val_score.view(-1, vocab_size), captions.view(-1))
                    running_loss += loss.item()
                    
                    for image in images:
                        val_pred = self.model.caption(image.unsqueeze(0), vocab)
                        
                        candidate = ' '.join(val_pred)

                        candidates[idx] = [candidate]
                        
                        label = all_labels[idx]

                        references[idx] = [label]
                    
                        idx += 1
                        
                val_loss = running_loss / len(val_loader)
                self.val_losses.append(val_loss)
                    
            metrics = Scorer(references, candidates).compute_scores()

            self.bleu1_scores.append(metrics['Bleu_1'])
            self.bleu2_scores.append(metrics['Bleu_2'])
            self.bleu3_scores.append(metrics['Bleu_3'])
            self.bleu4_scores.append(metrics['Bleu_4'])

            cumulative_bleu_score = (metrics['Bleu_1'] + metrics['Bleu_2'] + metrics['Bleu_3'] + metrics['Bleu_4']) / 4
            self.cumulative_bleu_scores.append(cumulative_bleu_score)

            self.cider_scores.append(metrics['CIDEr'])

            self.rougel_scores.append(metrics['ROUGE_L'])

            self.meteor_scores.append(metrics['METEOR'])

            self._log(epoch)

        if self.save_path:
            self.save_weights(self.save_path)

    def get_train_losses(self):
        return self.train_losses
    
    def get_val_losses(self):
        return self.val_losses
    
    def get_scores(self):
        scores = {
                "BLEU 1": self.bleu1_scores,
                "BLEU 2": self.bleu2_scores,
                "BLEU 3": self.bleu3_scores,
                "BLEU 4": self.bleu4_scores,
                "Cumulative BLEU": self.cumulative_bleu_scores,
                "METEOR": self.meteor_scores,
                "ROUGE_L": self.rougel_scores,
                "CIDEr": self.cider_scores,
                }
        
        return scores
    
    def save_weights(self, path):
        torch.save(self.model.state_dict(), path)

    def _log(self, epoch):
        log = ""
        log += f" ---------------------------------------------------------------------------------------------------\n"
        log += f"|     Epoch [{epoch + 1}/{self.num_epochs}]     |         Training Loss: {self.train_losses[-1]: .6f}         |      Validation Loss: {self.val_losses[-1]: .6f}    |\n"
        log += f" ---------------------------------------------------------------------------------------------------\n"
        log += f"|        Cumulative BLEU Score: {self.cumulative_bleu_scores[-1]: .6f}       |               CIDEr Score: {self.cider_scores[-1]: .6f}                |\n"
        log += f" ---------------------------------------------------------------------------------------------------\n"
        log += f"|        METEOR Score: {self.meteor_scores[-1]: .6f}                |               ROUGE_L Score: {self.rougel_scores[-1]: .6f}              |\n"
        log += f" ---------------------------------------------------------------------------------------------------\n"

        print(log)
        

# Training and Evaluating: EncoderDecoder Model = VGG16 + Vanilla RNN

## Setting Hyperparameters

In [ ]:
num_epochs = 4
enc_dim = 2048
embed_size = 224
hidden_size = 512
num_layers = 1
learning_rate = 0.001

## Configuring The Model

In [ ]:
encoder = Encoder(vgg16(weights=VGG16_Weights.IMAGENET1K_FEATURES), embed_size).to(device)
decoder = Decoder(nn.RNN, embed_size, vocab_size, hidden_size, num_layers, attention=False).to(device)

model = EncoderDecoder(encoder, decoder).to(device)

In [ ]:
print("\n--------------------------------------------------\n")
print("Encoder Model Summary:")
summary(model.encoder, (3, 224, 224))
print("\n--------------------------------------------------\n\n")

## Training The Model

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = vocab.stoi["<PAD>"]).to(device)

optimizer = optim.Adam(model.parameters(), lr = learning_rate)

save_path = "/kaggle/working/EncoderDecoder_VGG16_RNN.pth"

trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, num_epochs, save_path)

trainer.train()

## Fetching Evaluation Metrics

In [ ]:
train_losses = trainer.get_train_losses()
val_losses = trainer.get_val_losses()

scores = trainer.get_scores()

bleu1_scores = scores["BLEU 1"]
bleu2_scores = scores["BLEU 2"]
bleu3_scores = scores["BLEU 3"]
bleu4_scores = scores["BLEU 4"]

cumulative_bleu_scores = scores["Cumulative BLEU"]

cider_scores = scores["CIDEr"]
meteor_scores = scores["METEOR"]
rougel_scores = scores["ROUGE_L"]

## Plotting Cross-Validation and Scores

In [ ]:
plt.figure(0)
plt.plot(train_losses, label = 'Training loss')
plt.plot(val_losses, label = 'Validation loss')
plt.ylabel('Cross Entropy Loss')
plt.legend()
plt.savefig(f'/kaggle/working/{freq_threshold}_{batch_size}_{hidden_size}_{num_epochs}_losses.png')

plt.figure(1)
plt.plot(bleu1_scores, label = 'BLEU 1')
plt.plot(bleu2_scores, label = 'BLEU 2')
plt.plot(bleu3_scores, label = 'BLEU 3')
plt.plot(bleu4_scores, label = 'BLEU 4')
plt.ylabel('BLEU Scores')
plt.legend()
plt.savefig(f'/kaggle/working/{freq_threshold}_{batch_size}_{hidden_size}_{num_epochs}_bleu_scores.png')
        
plt.figure(2)
plt.plot(cumulative_bleu_scores, label = 'Cumulative BLEU')
plt.plot(cider_scores, label = 'CIDEr')
plt.plot(meteor_scores, label = 'METEOR')
plt.plot(rougel_scores, label = 'ROUGE_L')
plt.ylabel('Scores')
plt.legend()
plt.savefig(f'/kaggle/working/{freq_threshold}_{batch_size}_{hidden_size}_{num_epochs}_scores.png')

plt.show()

## *Loading Pre-Trained Parameters

In [ ]:
load_path = "/kaggle/working/EncoderDecoder_VGG16_RNN.pth"

model.load_state_dict(torch.load(load_path))
model.eval()

## Inference

In [ ]:
for _ in range(10):
    idx = np.random.randint(0, data_size - 1)

    image, _ = dataset[idx]

    prediction = ' '.join(model.caption(image.unsqueeze(0), vocab))

    image = image.permute(1,2,0)
    
    plt.title(prediction)
    plt.imshow(image)

    plt.show()    